In [10]:
! pip install classification-models-3D

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 2.5 MB/s eta 0:00:00


In [11]:
!pip install keras_applications

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nibabel as nib
from tensorflow import keras
import tensorflow_addons as tfa
import random
from scipy import ndimage
from tensorflow import keras
import os

In [54]:
df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")

In [55]:
n_image_list = np.arange(0, df.shape[0])
random_n_image_list = np.random.choice(n_image_list, df.shape[0], replace=False)

In [56]:
def data_gen():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in random_n_image_list[:1800]:
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
            yield ({"input_1": ct}, {'co': df.iloc[i]["patient_overall"], 
                                    'c1': df.iloc[i]["C1"],
                                    'c2': df.iloc[i]["C2"],
                                    'c3': df.iloc[i]["C3"],
                                    'c4': df.iloc[i]["C4"],
                                    'c5': df.iloc[i]["C5"],
                                    'c6': df.iloc[i]["C6"],
                                    'c7': df.iloc[i]["C7"],})
        except:
            continue

            
            
def test_gen():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in random_n_image_list[1800:]:
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
            yield ({"input_1": ct}, {'co': df.iloc[i]["patient_overall"], 
                                    'c1': df.iloc[i]["C1"],
                                    'c2': df.iloc[i]["C2"],
                                    'c3': df.iloc[i]["C3"],
                                    'c4': df.iloc[i]["C4"],
                                    'c5': df.iloc[i]["C5"],
                                    'c6': df.iloc[i]["C6"],
                                    'c7': df.iloc[i]["C7"],})
        except:
            continue

In [57]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,60])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

testset = tf.data.Dataset.from_generator(
     test_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,60])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

In [58]:
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-180, -10, -5, 5, 10]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume

In [59]:
def train_preprocessing(volume, label):

#     volume["input_1"] = rotate(volume["input_1"])
    volume["input_1"] = tf.expand_dims(volume["input_1"], axis=3)
    return volume, label

def test_preprocessing(volume, label):

    volume["input_1"] = tf.expand_dims(volume["input_1"], axis=3)
    return volume, label

In [60]:
dataset = (
    dataset.map(train_preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(10)
    .prefetch(1)
)

testset = (
    testset.map(test_preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(10)
    .prefetch(1)
)

In [19]:
from classification_models_3D.tfkeras import Classifiers
eff, preprocess_input = Classifiers.get('resnet34')
eff = eff(input_shape=(100, 100, 60, 1), weights=None, include_top=False)

In [20]:
inputs = keras.Input((100, 100, 60, 1), name = "input_1")

x = eff(inputs)

x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.Dense(16, activation="relu")(x)
x = keras.layers.Dense(8, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)

outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(x)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(x)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(x)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(x)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(x)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(x)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(x)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(x)
                                               
model = keras.models.Model(inputs=inputs, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7])

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 60 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 4, 4, 2, 512) 63487427    input_1[0][0]                    
__________________________________________________________________________________________________
flatten (Flatten)               (None, 16384)        0           model[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16384)        65536       flatten[0][0]                    
____________________________________________________________________________________________

In [62]:
opt = tfa.optimizers.AdamW(
    weight_decay=2e-4,
    learning_rate = 0.0002,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.1, 
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=opt,
              loss={"co": loss, 
                   "c1": loss,
                   "c2": loss,
                   "c3": loss,
                   "c4": loss,
                   "c5": loss,
                   "c6": loss,
                   "c7": loss,},
                metrics={"co": metrics, 
                   "c1": metrics,
                   "c2": metrics,
                   "c3": metrics,
                   "c4": metrics,
                   "c5": metrics,
                   "c6": metrics,
                   "c7": metrics,})


In [23]:
!mkdir best_model

In [63]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './best_model/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [ ]:
history = model.fit(dataset, validation_data = testset, epochs=25, callbacks=[model_checkpoint_callback_LASSO])

Epoch 1/25


2022-10-07 08:53:13.076422: I tensorflow/core/grappler/optimizers/generic_layout_optimizer.cc:345] Cancel Transpose nodes around Pad: transpose_before=model_1/model/bn_data/FusedBatchNormV3-0-1-TransposeNCDHWToNDHWC-LayoutOptimizer pad=model_1/model/zero_padding3d/Pad transpose_after=model_1/model/conv0/Conv3D-0-TransposeNDHWCToNCDHW-LayoutOptimizer,gradient_tape/model_1/model/conv0/Conv3D/Conv3DBackpropFilterV2-0-TransposeNDHWCToNCDHW-LayoutOptimizer,gradient_tape/model_1/model/conv0/Conv3D/Shape-0-TransposeNDHWCToNCDHW-LayoutOptimizer


    180/Unknown - 75s 372ms/step - loss: 3.4578 - co_loss: 0.6695 - c1_loss: 0.3522 - c2_loss: 0.4586 - c3_loss: 0.2838 - c4_loss: 0.3182 - c5_loss: 0.3720 - c6_loss: 0.4667 - c7_loss: 0.5367 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.7707

2022-10-07 08:54:22.693541: I tensorflow/core/grappler/optimizers/generic_layout_optimizer.cc:345] Cancel Transpose nodes around Pad: transpose_before=model_1/model/bn_data/FusedBatchNormV3-0-0-TransposeNCDHWToNDHWC-LayoutOptimizer pad=model_1/model/zero_padding3d/Pad transpose_after=model_1/model/conv0/Conv3D-0-TransposeNDHWCToNCDHW-LayoutOptimizer


180/180 [==============================] - 82s 412ms/step - loss: 3.4578 - co_loss: 0.6695 - c1_loss: 0.3522 - c2_loss: 0.4586 - c3_loss: 0.2838 - c4_loss: 0.3182 - c5_loss: 0.3720 - c6_loss: 0.4667 - c7_loss: 0.5367 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.7707 - val_loss: 56.6728 - val_co_loss: 29.8883 - val_c1_loss: 2.7975 - val_c2_loss: 10.8237 - val_c3_loss: 0.3146 - val_c4_loss: 1.3225 - val_c5_loss: 1.8055 - val_c6_loss: 3.0974 - val_c7_loss: 6.6233 - val_c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.4862
Epoch 2/25
180/180 [==============================] - 72s 401ms/step - loss: 3.4501 - co_loss: 0.6626 - c1_loss: 0.3528 - c2_loss: 0.4545 - c3_loss: 0.2848 - c4_loss: 0.3199 - c5_loss: 0.3721 - c6_loss: 0.4673 - c7_loss: 0.5361 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.7743 - val_loss: 4.1453 - val_co_loss: 1.0956 - val_c1_loss: 0.4048 - val_c2_loss: 0.6359 - val_c3_loss: 0.2999 - val_c4_loss: 0.3425 - val_c5_loss: 0.3727 - val_c6_loss: 0.4249 - val_c7_loss: 0.5689 - val_c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0

In [16]:
model.load_weights("./best_model/model").expect_partial()

In [ ]:
!mkdir final_model

In [ ]:
model.save('./final_model/model')